# Process pseudomonas data
This notebook trains a VAE on the PAO1 and PA14 RNA-seq compendia

1. Selects template experiment from the compendium
2. Normalizes the gene expression data from the RNA-seq Pseudomonas compendium
3. Train VAE on the normalized data

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle

from ponyo import utils, train_vae_modules
from generic_expression_patterns_modules import process

Using TensorFlow backend.
/home/alexandra/anaconda3/envs/generic_expression_new/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# Set seeds to get reproducible VAE trained models
process.set_all_seeds()

### Set parameters for data processing

Most parameters are read from `config_filename`. We manually selected bioproject [GEOD-33245](https://www.ebi.ac.uk/arrayexpress/experiments/E-GEOD-33245/?s_sortby=col_8&s_sortorder=ascending), as the template experiment, which contains multiple different comparisons including WT vs *crc* mutants, WT vs *cbr* mutants in different conditions.

In [3]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

# Read in config variables
config_filename = "config_pseudomonas_pa14_rnaseq.tsv"

params = utils.read_config(config_filename)

local_dir = params["local_dir"]
dataset_name = params["dataset_name"]

# Column header containing sample ids
metadata_colname = params["metadata_colname"]

# Template experiment ID
project_id = params["project_id"]

# Output file: pickled list of shared genes(generated during gene ID mapping)
shared_genes_filename = params["shared_genes_filename"]

# Output files of pseudomonas template experiment data
raw_template_filename = params["raw_template_filename"]
processed_template_filename = params["processed_template_filename"]

# Output files of pseudomonas compendium data
# raw_compendium_filename = params['raw_compendium_filename']
processed_compendium_filename = params["processed_compendium_filename"]
normalized_compendium_filename = params["normalized_compendium_filename"]

# Output file: pickled scaler (generated during compendium normalization)
scaler_filename = params["scaler_filename"]

# Load metadata file with mapping between experiments and associated samples
metadata_filename = "data/metadata/SraRunTable.csv"
metadata_delimiter = ","
experiment_id_colname = "SRA_study"

### Normalize compendium
The compendium is MR normalized, but here we will 0-1 normalize it

In [4]:
process.normalize_compendium(
    processed_compendium_filename,
    normalized_compendium_filename,
    scaler_filename,
)

input: dataset contains 576 samples and 5887 genes


## Get raw pseudomonas template experiment

In [5]:
process.process_raw_template_pseudomonas(
    processed_compendium_filename,
    project_id,
    metadata_filename,
    metadata_delimiter,
    experiment_id_colname,
    metadata_colname,
    raw_template_filename,
)

In [6]:
test2 = pd.read_csv(raw_template_filename, sep="\t", index_col=0, header=0)

In [7]:
test2.head()

,PA14_55610,PA14_55600,PA14_55590,PA14_55580,PA14_55570,PA14_55560,PA14_55550,PA14_55540,PA14_55530,PA14_55520,...,PA14_19205,PA14_17675,PA14_67975,PA14_36345,PA14_43405,PA14_38825,PA14_24245,PA14_28895,PA14_55117,PA14_59845
SRX1740204,179.103190,76.046134,33.867541,1142.769777,356.544170,104.303714,31.166448,85.188292,75.838358,5.402184,...,48.619660,19.738751,261.382614,275.719181,0.623329,56.515160,135.054610,156.871124,63.164002,261.382614
SRX1740205,160.820796,65.859945,31.972706,1395.886215,435.939228,115.829263,37.333399,101.470264,58.201812,4.786333,...,45.757345,25.654746,389.798976,251.952580,0.957267,116.212170,125.593383,164.075502,58.776172,296.561205
SRX1740206,156.853998,69.081551,34.540776,695.749907,207.394180,56.820323,20.634749,52.334508,55.325052,4.037234,...,38.877063,23.326238,253.897129,341.220995,1.644799,126.350456,133.826814,136.368776,56.521269,306.829746
SRX1740207,230.669314,54.576966,20.524671,474.399781,174.692938,39.183463,15.393503,47.346684,74.635167,5.364403,...,20.058201,24.722899,173.060294,483.029473,0.466470,113.352160,197.083488,102.156885,72.069583,390.901688
SRX1740208,199.661033,64.058640,32.725610,562.776041,169.372437,59.010541,19.147963,52.743935,86.165834,8.529547,...,27.677510,21.584977,165.890989,368.337180,1.218507,95.043526,143.435650,125.506194,71.717825,330.911616


In [8]:
test2.shape

(6, 5887)

**Note:**
* We are training our VAE model using ALL the data in the compendium.
* The template experiment is using a subset of the samples in the real experiment and using those in the DE analysis in order to ensure the comparison of samples with consistent backgrounds (i.e. some experiments have samples with 3 different biological conditions and for now our statistical test is doing a binary comparison).
* Simulated experiments are generated by shifting the template experiment (using ALL samples in the real experiment) in the latent space. Then dropping the samples to match the template experiment and perform DE analysis.


So there is an inconsistency in the samples used to learn a low-dimensional representation and those used to calculate DE statistics. This inconsistency should not not change the simulated experiments since all samples in the template experiment are moved the same amount in the latent space. The only way for this inconsistency to effect the simulated experiments is if the low dimensional space is significantly different including all the experiment samples vs only including a subset. However, we believe that such few samples will likely not effect the space. Furthermore, the dataset used to train the VAE should be a general representation of gene expression patterns and shouldn't have to be include the template experiment.

### Train VAE

In [9]:
# Create VAE directories if needed
output_dirs = [
    os.path.join(base_dir, dataset_name, "models"),
    os.path.join(base_dir, dataset_name, "logs"),
]

NN_architecture = params["NN_architecture"]

# Check if NN architecture directory exist otherwise create
for each_dir in output_dirs:
    sub_dir = os.path.join(each_dir, NN_architecture)
    os.makedirs(sub_dir, exist_ok=True)

In [10]:
# Train VAE on new compendium data
train_vae_modules.train_vae(config_filename, normalized_compendium_filename)

input dataset contains 576 samples and 5887 genes
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
tracking <tf.Variable 'Variable:0' shape=() dtype=float32> beta
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/alexandra/anaconda3/envs/generic_expression_new/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output custom_variational_layer_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to custom_variational_layer_1.
  'be expecting any data to be passed to {0}.'.format(name))



Train on 432 samples, validate on 144 samples
Epoch 1/120
432/432 [==============================] - 27s 63ms/step - loss: 2283.0599 - kl_loss: 14.4142 - recons_loss: 2283.0598 - val_loss: 2072.2133 - val_kl_loss: 11.3726 - val_recons_loss: 2072.2131
Epoch 2/120
432/432 [==============================] - 22s 50ms/step - loss: 2023.2903 - kl_loss: 14.0545 - recons_loss: 2023.2904 - val_loss: 2023.5950 - val_kl_loss: 12.1395 - val_recons_loss: 2023.5951
Epoch 3/120
432/432 [==============================] - 22s 50ms/step - loss: 2006.5272 - kl_loss: 14.4879 - recons_loss: 2006.5271 - val_loss: 2021.7129 - val_kl_loss: 13.7628 - val_recons_loss: 2021.7129
Epoch 4/120
432/432 [==============================] - 22s 50ms/step - loss: 1992.3631 - kl_loss: 14.8924 - recons_loss: 1992.3628 - val_loss: 1981.2521 - val_kl_loss: 14.1386 - val_recons_loss: 1981.2522
Epoch 5/120
432/432 [==============================] - 23s 53ms/step - loss: 1979.5231 - kl_loss: 15.2980 - recons_loss: 1979.5228 - 

432/432 [==============================] - 22s 51ms/step - loss: 1852.6875 - kl_loss: 37.7153 - recons_loss: 1852.6875 - val_loss: 1859.3757 - val_kl_loss: 31.8551 - val_recons_loss: 1859.3756
Epoch 40/120
432/432 [==============================] - 23s 52ms/step - loss: 1851.0299 - kl_loss: 38.3632 - recons_loss: 1851.0299 - val_loss: 1850.9495 - val_kl_loss: 32.7907 - val_recons_loss: 1850.9497
Epoch 41/120
432/432 [==============================] - 23s 54ms/step - loss: 1847.6412 - kl_loss: 39.2296 - recons_loss: 1847.6412 - val_loss: 1858.2139 - val_kl_loss: 35.1827 - val_recons_loss: 1858.2140
Epoch 42/120
432/432 [==============================] - 23s 53ms/step - loss: 1843.7412 - kl_loss: 40.1660 - recons_loss: 1843.7413 - val_loss: 1850.4691 - val_kl_loss: 35.6316 - val_recons_loss: 1850.4690
Epoch 43/120
432/432 [==============================] - 23s 54ms/step - loss: 1843.4530 - kl_loss: 41.0516 - recons_loss: 1843.4530 - val_loss: 1852.9183 - val_kl_loss: 36.3757 - val_recons

Epoch 79/120
432/432 [==============================] - 22s 51ms/step - loss: 1799.2623 - kl_loss: 68.1498 - recons_loss: 1799.2623 - val_loss: 1823.7103 - val_kl_loss: 57.4227 - val_recons_loss: 1823.7103
Epoch 80/120
432/432 [==============================] - 23s 53ms/step - loss: 1801.4118 - kl_loss: 69.0023 - recons_loss: 1801.4117 - val_loss: 1830.0215 - val_kl_loss: 60.0030 - val_recons_loss: 1830.0215
Epoch 81/120
432/432 [==============================] - 23s 54ms/step - loss: 1802.4660 - kl_loss: 69.6878 - recons_loss: 1802.4659 - val_loss: 1822.0183 - val_kl_loss: 61.7065 - val_recons_loss: 1822.0184
Epoch 82/120
432/432 [==============================] - 23s 54ms/step - loss: 1797.2242 - kl_loss: 70.5025 - recons_loss: 1797.2240 - val_loss: 1821.7182 - val_kl_loss: 63.6101 - val_recons_loss: 1821.7183
Epoch 83/120
432/432 [==============================] - 24s 54ms/step - loss: 1798.1611 - kl_loss: 71.2429 - recons_loss: 1798.1611 - val_loss: 1821.3030 - val_kl_loss: 62.6684

Epoch 119/120
432/432 [==============================] - 22s 51ms/step - loss: 1777.0869 - kl_loss: 98.4098 - recons_loss: 1777.0867 - val_loss: 1810.1967 - val_kl_loss: 86.0743 - val_recons_loss: 1810.1968
Epoch 120/120
432/432 [==============================] - 22s 51ms/step - loss: 1777.8798 - kl_loss: 99.1303 - recons_loss: 1777.8799 - val_loss: 1813.2892 - val_kl_loss: 86.9456 - val_recons_loss: 1813.2893
